**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [105]:
import skvideo
skvideo.setFFmpegPath("C:\\FFmpeg\\bin")
import keras
import numpy as np
import io
import base64
from IPython.display import HTML   
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense,Dropout, Activation
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,MaxPooling2D


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

(you can use Markdown and Latex)

Epsilon is defined as the probability of exploration and it consists in picking a random action among the possible ones. This so the agent will try new actions during the training, actions that may lead to a better cumulative reward. We need epsilon to be between 0 and 1, so that we have both exploration and exploitation: if it is equal to 1, the agent will always explore and never act greedily with respect to the action-value function. If it is 0, the agent will never explore and this may lead to overfitting (making actions too biased by the past during the training phase).


What the function act does is essentially this: it gives the agent a probability equal to epsilon of acting randomly (it chooses a number between 0 and 4, 4 excluded cause we have 4 possible directions) therefore exploring the environment and gathering new information, and it gives it also a probability of 1-epsilon of exploiting i.e. making the best decision given current
information at state s.


To understand why using epsilon is important, let's imagine a situation without exploration where the agent is stuck on a grid and it greedily chooses to move toward the boundary of the game and this results in it remaining in the same grid forever.
Therefore, keeping an explorative/stochastic element in its policy is useful to avoid such situations.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=45 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position simply states where the animal can/can't go and where it is: -1 indicates the boundaries where it cannot go, 0 represents the authorized positions (as this array is initialized as an array of zeros) and 1 the current position. 
The board instead, contains the rewards for each state. If the cell is (1,1), the rat will get the reward board[1,1]


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action)
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action. this time we need train=false cause we're testing and we don't need the epsilon-greedy approach
            action = agent.act(state,train=False)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/8.0. Average score (0.0)
Win/lose count 11.0/9.0. Average score (1.0)
Win/lose count 10.0/13.0. Average score (-0.3333333333333333)
Win/lose count 10.0/21.0. Average score (-3.0)
Win/lose count 11.5/11.0. Average score (-2.3)
Win/lose count 11.0/17.0. Average score (-2.9166666666666665)
Win/lose count 12.5/19.0. Average score (-3.4285714285714284)
Win/lose count 10.0/18.0. Average score (-4.0)
Win/lose count 11.0/12.0. Average score (-3.6666666666666665)
Win/lose count 12.0/15.0. Average score (-3.6)
Final score: -3.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




>1)We start from:
    
%%latex
\begin{equation}
Q^{\pi}(s,a)=E[\sum_{k=0}^{\infty} \gamma^{k}r_{t+k}|s_{t}=s,a_{t}=a]
\end{equation}
>then we take out the case when t=0 and iterate



$$= E[r(s,a)+\sum_{k=1}^{\infty} \gamma^{k}r_{t+k}|s_{t}=s,a_{t}=a] \\\\
=r(s,a)+\gamma E[E[\sum_{k=0}^{\infty} \gamma^{k}t_{t+k+1|s_{t+1}=s',a_{t+1}=a'}]|s_{t}=s,a_{t}=a]\\\\
= r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_{t} = s, a_{t} = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a'] \\\\
=  \sum_{(s',a')} (p(s_{t+1}=s',a_{t+1}=a' |s_{t} = s, a_{t} = a ) r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_{t} = s, a_{t} = a )E[\sum_{k=0}^{\infty}\gamma^{k} r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\\\
=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

>2) Then for the second equivalence we have that:

$$Q^{*}(s,a) = \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\\\
= \max_{\pi}r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_{t} = s, a_{t} = a )Q^{\pi}(s',a')\\\\
= r(s,a) + \gamma \max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \qquad \text{with } \pi=(a',\pi')$$

>At this point we should exchange the maximum and the sum and in order to do so, we have to consider the following inequality:


$$\max_{\pi} \sum_{(s')} p(s_{t+1}=s'|s_{t} = s, a_{t}= a) Q^{\pi'}(s',a') 
\leq \sum_{(s',a')} p(s_{t+1}=s'|s_{t} = s, a_{t} = a) \max_{\pi}Q^{\pi'}(s',a')$$

>Then we have that 

$$\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')
\sum_{(s'} p(s_{t+1}=s'|s_{t} = s, a_{t} = a ) \max_{\pi'}Q^{\pi'}(s',a')\\\\ = \sum_{(s')} p(s_{t+1}=s'|s_{t} = s, a_{t} = a ) Q^{\hat{\pi}(s',a')}(s',a')
\leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s'|s_{t} = s, a_{t} = a)Q^{\pi'}(s',a')$$
 

>Hence we can rewrite our initial equation and find the final outcome:

$$Q^{*}(s,a)= r(s,a) + \gamma \sum(p(s_{t+1}=s'|s_{t} = s, a_{t} = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a')\\\\
= r(s,a) + \gamma \sum(p(s_{t+1}=s'|s_{t} = s, a_{t} = a )\max_{a'} Q^{*}(s',a')\\\\
=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'}Q^{*}(s',a')]$$

>3) Finally we have to deduce that the given objective function is possible.

%%latex
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+ \gamma \max_{a'}Q(s',a',\theta)-Q(s,a,\theta) \Vert^{2}
\end{equation*}

With this loss function we try to force the Q value to verify the well known Bellman equation.
Furthermore, we can derive the given objective by minimization of this loss function:

%%latex
\begin{equation}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}(( Q^* (s,a,\theta ) - Q(s,a,\theta)) ^2)
\end{equation}

\begin{equation}
\mathcal{L}(\theta) =  E_{s' \sim \pi^*(.|s,a)} (( r(s,a)+\gamma \max_{a'}Q^{*}(s',a', \theta) - Q(s,a,\theta)) ^2)
\end{equation}
\begin{equation}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma \max \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}
\end{equation}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [35]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]

    def random_access(self):
        i=np.random.randint(0,len(self.memory))
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [75]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [128]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])
    

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        
        for i in range(self.batch_size):
            #considering new information in order to make new predictions
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l
   
    
    
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu')) #let's start with one layer
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model


In [36]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0060 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/050 | Loss 0.0122 | Win/lose count 3.5/6.0 (-2.5)
Epoch 002/050 | Loss 0.0031 | Win/lose count 3.5/8.0 (-4.5)
Epoch 003/050 | Loss 0.0083 | Win/lose count 1.0/4.0 (-3.0)
Epoch 004/050 | Loss 0.0107 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/050 | Loss 0.0047 | Win/lose count 5.0/9.0 (-4.0)
Epoch 006/050 | Loss 0.0372 | Win/lose count 2.0/2.0 (0.0)
Epoch 007/050 | Loss 0.0047 | Win/lose count 3.5/5.0 (-1.5)
Epoch 008/050 | Loss 0.0087 | Win/lose count 2.5/1.0 (1.5)
Epoch 009/050 | Loss 0.0125 | Win/lose count 7.5/3.0 (4.5)
Epoch 010/050 | Loss 0.0146 | Win/lose count 2.5/0 (2.5)
Epoch 011/050 | Loss 0.0161 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/050 | Loss 0.0039 | Win/lose count 0.5/0 (0.5)
Epoch 013/050 | Loss 0.0035 | Win/lose count 5.5/8.0 (-2.5)
Epoch 014/050 | Loss 0.0054 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/050 | Loss 0.0015 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/050 | Loss 0.0519 | Win/lose count 2.5/2.0

# COMPARISON OF DIFFERENT ARCHITECTURES (ALWAYS FULLY CONNECTED) AND PARAMETERS (learning rate, batch size or memory size)

>One clear problem coming from the above structure is that it does not provide much space for exploration, as we can see in the video: the agent reaches a corner and find it difficult to move away. We'll deal with this problem later in the project.

| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
   1 layer (30 neurons)| 25      |     0.1     |         0.1 |32 |2000 |0.46|\n",
   1 layer (50 neurons)| 25      |     0.1     |         0.1 |32 |2000 |1.24|\n",
   1 layer (200 neurons)| 25      |     0.1     |         0.1 |32 |2000 |-5.5|\n",
   
   
>it looks addin more neurons is not helpful so we're gonna stik to 30

>Now let's try and change the architecture

| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
   2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |2000 |1.86|\n",
   3 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |2000 |-1.38|\n",
   
   
>it looks like increasing the number of layers is not helping the mean score. so let's stick with 2 layers and try adding some dropout. I'm gonna try with 3 different dropout rates [0.2,0.5,0.7]. In this case the best combination is with a dropout of 0.2

| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
   2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |2000 |0.5|\n",
   
 >Now I'm gonna use the last best architecture (2 layers with BatchNorm and without dropout) and change the learning rate in the following range:learning_rates= [0.01, 0.05, 0.1, 0.2]
 
 
| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
   2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.01 |32 |2000 |-0.46|\n",
    2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.05 |32 |2000 |-0.92 |\n",
     2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |2000 |1.86|\n",
      2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.2 |32 |2000 | -0.48|\n",
      
      
>Now again, I'm gonna use the last best architecture (2 layers with BatchNorm and without dropout, lr=0.1) and change the memory size in the following range [1000,2000,3000]

 
| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
   2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |1000 |0.12|\n",
   2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |2000 |1.86|\n",
   2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |3000 |-0.46|\n",
   
 >Finally the last thing to check is the batch size, in the range [10,32,64]
 
| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
  2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |10 |2000 |-1.12 |\n",
  2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |32 |2000 |1.86|\n",
  2 layer (30 neurons)+ BatchNorm| 25      |     0.1     |         0.1 |64 |2000 |1.38|\n",
 
 
The most evident thing is that there is some stochasticity in the numbers so it's not easy to determine strictly which parameters actually contribute a lot (positively or negatively). However we can see that the learning rate, as well as the memory size, should not be too high, while for the batch size, a value superior to 32 still provides good results.
Eventually, the best architecture was one with 2 layers and an output of 30 neurons+ batch normalization, with epsilon=0.1,
lr=0.1, batch size=32, mekmory size=2000, that provided a final score of 1.86

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [133]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        model.add(Conv2D(60,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [132]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/025 | Loss 0.0162 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/025 | Loss 0.0062 | Win/lose count 1.5/1.0 (0.5)
Epoch 002/025 | Loss 0.0336 | Win/lose count 3.5/6.0 (-2.5)
Epoch 003/025 | Loss 0.0030 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/025 | Loss 0.0010 | Win/lose count 1.0/6.0 (-5.0)
Epoch 005/025 | Loss 0.0300 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/025 | Loss 0.0054 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/025 | Loss 0.0031 | Win/lose count 1.5/2.0 (-0.5)
Epoch 008/025 | Loss 0.0111 | Win/lose count 1.5/3.0 (-1.5)
Epoch 009/025 | Loss 0.0008 | Win/lose count 2.0/1.0 (1.0)
Epoch 010/025 | Loss 0.0030 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/025 | Loss 0.0059 | Win/lose count 3.0/5.0 (-2.0)
Epoch 012/025 | Loss 0.0108 | Win/lose count 9.5/3.0 (6.5)
Epoch 013/025 | Loss 0.0029 | Win/lose count 3.5/3.0 (0.5)
Epoch 014/025 | Loss 0.0491 | Win/lose count 7.5/3.0 (4.5)
Epoch 015/025 | Loss 0.0021 | Win/lose count 5.5/4.0 (1.5)
Epoch 016/025 | Loss 0.0971 | Win/lose count 12.

# COMPARISON OF DIFFERENT ARCHITECTURES (CNN)

>We can immediately notice that the final score coming from the above architecture is equal to 7.08 which is already much higher than
any score we obtained before using only fully connected layers (not a surprise).
>In the previous case (fully connected layers), we saw that some good parameters were: lr=0.1, batch size=32 and memory size=2000, so we're gonna start from those, and change the structure of the network.

 
| Architecture       |     Epochs    |   Epsilon parameter | Learning Rate | Batch size|Memory size| Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
2 layers (60 and 30 nn)| 25      |     0.1     |         0.1 |32 |2000 |7.08|\n",
   2 layers (60 and 30 nn)+ MaxPooling2D(2,2)| 25      |     0.1     |         0.1 |32 |2000 |1.62|\n",
   2 layers (60 and 30 nn)+ Dropout(0.2)| 25      |     0.1     |         0.1 |32 |2000 |4.2 |\n",
   2 layers (60 and 30 nn)+ Dropout(0.5)| 25      |     0.1     |         0.1 |32 |2000 | 3.44|\n",
   2 layers (60 and 30 nn)+Adam opt| 25      |     0.1     |         0.1 |32 |2000 |-3.98 |\n",
   
   
>We can see that the best architectures is the first one with 2 convolutional layers and the sgd optimizer.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [135]:
epochs_test=10
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.0/1.0. Average score (1.0)
Win/lose count 3.5/0. Average score (1.625)
Win/lose count 2.0/1.0. Average score (1.5)
Win/lose count 0.5/0. Average score (1.3333333333333333)
Win/lose count 0.5/0. Average score (1.2142857142857142)
Win/lose count 2.0/0. Average score (1.3125)
Win/lose count 0/0. Average score (1.1666666666666667)
Win/lose count 4.0/1.0. Average score (1.35)
Final score: 1.35
Test of the FC
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 1.5/4.0. Average score (-0.75)
Win/lose count 0/0. Average score (-0.5)
Win/lose count 1.0/2.0. Average score (-0.625)
Win/lose count 1.0/2.0. Average score (-0.7)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 1.0/5.0. Average score (-1.0)
Win/lose count 1.5/1.0. Average score (-0.8125)
Win/lose count 0/0. Average score (-0.7222222222222222)
Win/lose count 0.5/0. Average score (-0.6)
Final score: -0.6


In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

>Both architectures show the same problem: the agent tends not to explore the entire area, but mainly sticks to a certain corner, which implies that we should try to improve the exploration policy.
>Now we're gonna try changing the temperature in the range [0.1,0.3,0.6,0.9] for the best architectures of the 2 cases.


| Architecture       |    Temperature    | Final Test Score (Average over 25 epochs)
| :------------ | :-------------: | -------------: | 
CNN|0.1 |1.06|\n",
CNN|0.3 | 7.08|\n",
CNN|0.6 |9.18 |\n",
CNN|0.9 | 26.64|\n",
FC|0.1 |-0.14|\n",
FC|0.3 |0.96|\n",
FC|0.6 |3.9|\n",
FC|0.9 |16.16|\n",


>We can see that the overall score using the CNN srchitecture is highly better than the one obtained using FC networks. THis gap is increasing with the temperature. This gap between the 2 scores increases with temperature.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [146]:

def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    #now we know that the problem here is the exploration, therefore we're gonna use the decay paramter epsilon
    #in order to try the first of the methods proposed above
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
        
    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #Here we add a new parameter in order to promote exploration, given that, as we said before,
        #we are facing the problem of a lack of exploration

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    


In [165]:
# Training
epochs_train = 25
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon =0.8, memory_size=2000, batch_size = 32,n_state=3) 
train_explore(agent, env, epochs_train,decay_parameter_epsilon=0.1, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))  #we can see that epsilon decreases with time


Epoch 000/025 | Loss 0.0139 | Win/lose count 9.0/27.60000000000008 (-18.60000000000008)
Epoch 001/025 | Loss 0.0138 | Win/lose count 10.5/22.600000000000026 (-12.100000000000026)
Epoch 002/025 | Loss 0.0132 | Win/lose count 8.5/26.800000000000033 (-18.300000000000033)
Epoch 003/025 | Loss 0.0062 | Win/lose count 15.0/19.00000000000001 (-4.000000000000011)
Epoch 004/025 | Loss 0.0059 | Win/lose count 10.0/26.400000000000066 (-16.400000000000066)
Epoch 005/025 | Loss 0.0041 | Win/lose count 20.5/15.599999999999971 (4.900000000000029)
Epoch 006/025 | Loss 0.0069 | Win/lose count 10.0/20.899999999999995 (-10.899999999999995)
Epoch 007/025 | Loss 0.0235 | Win/lose count 16.5/23.90000000000006 (-7.400000000000059)
Epoch 008/025 | Loss 0.0053 | Win/lose count 14.5/22.10000000000003 (-7.60000000000003)
Epoch 009/025 | Loss 0.0069 | Win/lose count 18.0/18.600000000000016 (-0.6000000000000156)
Epoch 010/025 | Loss 0.0058 | Win/lose count 20.5/17.999999999999975 (2.500000000000025)
Epoch 011/025 

In [166]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')



Win/lose count 11.0/0. Average score (11.0)
Win/lose count 19.5/0. Average score (15.25)
Win/lose count 5.5/0. Average score (12.0)
Win/lose count 20.5/4.0. Average score (13.125)
Win/lose count 18.0/0. Average score (14.1)
Win/lose count 16.5/1.0. Average score (14.333333333333334)
Win/lose count 19.5/0. Average score (15.071428571428571)
Win/lose count 12.5/0. Average score (14.75)
Win/lose count 20.0/0. Average score (15.333333333333334)
Win/lose count 21.0/1.0. Average score (15.8)
Final score: 15.8


In [ ]:
HTML(display_videos('cnn_test_explore10.mp4'))

>As we can see from the video, this approach widely improves the outcome: the agent is no more stuck in a corner of the map.


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [167]:
class DQN_Mimicks(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_Mimicks, self).__init__(epsilon = epsilon)
        # Discount value
        self.discount = 0.99
        self.grid_size = grid_size
        self.n_state = n_state
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) #here we're just gonna mimick the expert's behaviour so we're considering state s and anction a
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 10 
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mimicks(DQN_Mimicks):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mimicks, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(60,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) 
        model.add(Conv2D(30,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) 
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [168]:
def train_mimicks(expert,agent,env,epoch,prefix=''): #same as before but we add expert
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    print('Final score: '+str(score/epoch))

In [169]:
epochs_train = 25
expert = agent
agent_n = DQN_CNN_Mimicks(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimicks(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')

Epoch 000/025 | Loss 0.6772 | Win/lose count 5.5/0 (5.5)
Epoch 001/025 | Loss 0.8755 | Win/lose count 4.0/0 (4.0)
Epoch 002/025 | Loss 2.4195 | Win/lose count 13.5/0 (13.5)
Epoch 003/025 | Loss 2.6044 | Win/lose count 26.5/2.0 (24.5)
Epoch 004/025 | Loss 4.9141 | Win/lose count 25.0/1.0 (24.0)
Epoch 005/025 | Loss 3.7443 | Win/lose count 10.0/1.0 (9.0)
Epoch 006/025 | Loss 3.3339 | Win/lose count 14.5/0 (14.5)
Epoch 007/025 | Loss 3.2899 | Win/lose count 19.0/0 (19.0)
Epoch 008/025 | Loss 3.5582 | Win/lose count 19.5/1.0 (18.5)
Epoch 009/025 | Loss 2.4768 | Win/lose count 19.5/1.0 (18.5)
Epoch 010/025 | Loss 4.8687 | Win/lose count 26.5/1.0 (25.5)
Epoch 011/025 | Loss 3.8920 | Win/lose count 18.0/0 (18.0)
Epoch 012/025 | Loss 4.9549 | Win/lose count 17.5/0 (17.5)
Epoch 013/025 | Loss 4.6699 | Win/lose count 16.0/0 (16.0)
Epoch 014/025 | Loss 4.6236 | Win/lose count 19.0/0 (19.0)
Epoch 015/025 | Loss 2.2386 | Win/lose count 17.5/0 (17.5)
Epoch 016/025 | Loss 5.0102 | Win/lose count 30.5

In [170]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')

Win/lose count 0.5/0. Average score (0.5)
Win/lose count 9.0/0. Average score (4.75)
Win/lose count 5.0/0. Average score (4.833333333333333)
Win/lose count 17.5/3.0. Average score (7.25)
Win/lose count 18.5/1.0. Average score (9.3)
Win/lose count 11.5/0. Average score (9.666666666666666)
Win/lose count 11.0/1.0. Average score (9.714285714285714)
Win/lose count 14.0/2.0. Average score (10.0)
Win/lose count 9.0/2.0. Average score (9.666666666666666)
Win/lose count 6.5/0. Average score (9.35)
Final score: 9.35


In [ ]:
HTML(display_videos('cnn_test_mimick10.mp4'))

>What we can see is that this approach performes better than the one without much exploration but as this is just an imitation of the expert, it is worse than our previous attempt where we managed to incorporate a decreasing epsilon-greedy exploration.

***